In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 이거 실행하고 다시 런타임 시작
# 한글 폰트
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 27 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 1s (6,912 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletyp

In [3]:
!pip install openpyxl
!pip install pykrx
!pip install opendartreader
!pip install --upgrade opendartreader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 8.7 MB/s 
     |████████████████████████████████| 52 kB 973 kB/s 
     |████████████████████████████████| 254 kB 44.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import pandas as pd
import numpy as np
from pykrx import stock
import re
pd.options.display.max_columns = None

# KisValue 데이터 가공
- KisValue 데이터에서 2018~2021 코스닥 재무제표 가져오기

In [5]:
df = pd.read_csv('/content/drive/MyDrive/핀테크 파이널/data/2018-2021_원본.csv')

In [6]:
df.head(10)

,Massive Download,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94,Unnamed: 95,Unnamed: 96,Unnamed: 97,Unnamed: 98,Unnamed: 99,Unnamed: 100,Unnamed: 101,Unnamed: 102,Unnamed: 103,Unnamed: 104,Unnamed: 105,Unnamed: 106,Unnamed: 107,Unnamed: 108,Unnamed: 109,Unnamed: 110,Unnamed: 111,Unnamed: 112,Unnamed: 113,Unnamed: 114,Unnamed: 115,Unnamed: 116,Unnamed: 117,Unnamed: 118,Unnamed: 119,Unnamed: 120,Unnamed: 121,Unnamed: 122,Unnamed: 123,Unnamed: 124,Unnamed: 125,Unnamed: 126,Unnamed: 127,Unnamed: 128,Unnamed: 129
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Portfolio - kosdaq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,2018/Annual,2018/Annual,2018/Annual,2018/Annual,2018/Annual,2018/Annual,2018/Annual,2018/Annual,2018/Annual,2018/Annual,2018/Annual,2018/Annual,2018/Annual,2018/Annual,2018/Annual,2018/Annual,2018/Annual,2018/Annual,2018/Annual,2018/Annual,2018/Annual,2018/Annual,2018/Annual,2018/Annual,2018/Ann

In [ ]:
#     # 총자본회전율
#     tmp['총자본회전율'] = (tmp['매출액'] / tmp['자산총계']) * 100

#     # 자기자본회전율
#     tmp['자기자본회전율'] = (tmp['매출액'] / tmp['자본총계']) * 100

## 위에 두개는 수치가 안맞는다.

In [7]:
df

,Massive Download,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94,Unnamed: 95,Unnamed: 96,Unnamed: 97,Unnamed: 98,Unnamed: 99,Unnamed: 100,Unnamed: 101,Unnamed: 102,Unnamed: 103,Unnamed: 104,Unnamed: 105,Unnamed: 106,Unnamed: 107,Unnamed: 108,Unnamed: 109,Unnamed: 110,Unnamed: 111,Unnamed: 112,Unnamed: 113,Unnamed: 114,Unnamed: 115,Unnamed: 116,Unnamed: 117,Unnamed: 118,Unnamed: 119,Unnamed: 120,Unnamed: 121,Unnamed: 122,Unnamed: 123,Unnamed: 124,Unnamed: 125,Unnamed: 126,Unnamed: 127,Unnamed: 128,Unnamed: 129
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Portfolio - kosdaq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,2018/Annual,2018/Annual,2018/Annual,2018/Annual,2018/Annual,2018/Annual,2018/Annual,2018/Annual,2018/Annual,2018/Annual,2018/Annual,2018/Annual,2018/Annual,2018/Annual,2018/Annual,2018/Annual,2018/Annual,2018/Annual,2018/Annual,2018/Annual,2018/Annual,2018/Annual,2018/Annual,2018/Annual,2018/Ann

In [9]:
# 재무제표 가공
df = pd.read_csv('/content/drive/MyDrive/핀테크 파이널/data/2018-2021_원본.csv')


def make_preprocessing_fs(df):

  # col_name = ['KIS', 'Stock', 'Name']
  # # kisvalue 데이터 다루기 편하게 handling
  # for i in df.columns[3:]:
  #   col_name.append(f"{df[i].iloc[5].split('/')[0]} ({df[i].iloc[4].split('/')[0]})")

  col_name = ['KIS', 'Stock', 'Name']
  for i in df.columns[3:]:
    col_name.append(f"{df[i].iloc[5]} ({df[i].iloc[4].split('/')[0]})")

  df.columns = col_name
  df.drop([0,1,2,3,4,5], axis=0, inplace=True)
  df = df.reset_index(drop=True)


  # 뒤에 널값인 데이터도 같이 들어왔는데 같이 삭제
  last_index = df[df['Name']=='(주)아바텍'].index # 데이터가 들어있는 맨마지막 인덱스. 이 이후꺼를 삭제시키면 된다.
  df = df.iloc[:list(df[df['Name']=='(주)아바텍'].index).pop()+1, :]
  
  
  # 모든 재무제표가 비어있는 코드 drop
  df = df.T
  fs_all_null_code = [] # 모든 재무제표가 null값인 코드 저장을 위해
  for i in df.columns:
    if df[i].isna().sum()==len(df)-3: # KIS, Stock, Name을 제외한 모든 재무제표 컬럼의 개수
      fs_all_null_code.append(i)
  df.drop(fs_all_null_code, axis=1, inplace=True)
  df


  df = df.T # 다시 돌리기
  df = df.reset_index(drop = True)

  
  # stock code 없는 '에스케이제5호기업인수목적(주)' 삭제
  del_index = list(df[df['Stock'].isnull()].index)
  df.drop(del_index, axis=0,inplace=True)
  df= df.reset_index(drop= True)


  # 인수합병된 기업들 빼기
  df['인수합병여부'] = 0
  df['인수합병여부'] = df.apply(lambda x: 1 if '인수목적' in x['Name'] else 0, axis=1)


  df = df[df['인수합병여부']==0]
  df.drop('인수합병여부', axis=1, inplace=True)
  df = df.reset_index(drop = True)




  # 컬럼 selection ->삭제할거 삭제하기
  del_list = ['영업이익증가율', '비용차감전순이익증가율', '순이익증가율', '종업원수증가율', '총자본법인세비용차감전순이익율', '자기자본법인세비용차감전순이익율', '자기자본순이익율', '자본금법인세비용차감전순이익율', '매출액법인세비용차감전순이익율', '매출액(종업원1인당)', '법인세비용차감전순이익(종업원1인당)','순이익(종업원1인당)', '인건비(종업원1인당)' , '평균발행주식수', '유형,임대주택자산감가상각비','무형자산상각비', '매출액/121000', '무형자산(계)', '자본총계/118900', '[보통주발행주식수]', '[우선주배당금]' , '보통주배당금', '영업이익(손실)',  '유형고정자산증가율', '유동자산증가율', '재고자산증가율', '2021']

  drop_col_list = []
  for i in del_list:
    for j in df.columns:
      if i in j:
        drop_col_list.append(j)
  

  df.drop(drop_col_list,axis=1,inplace=True)

  # 컬럼명 다시 깔끔하게 만들기
  col_new = ['KIS', 'Stock', 'Name']
  for i in df.columns[3:]:
    start_index = i.find('/')
    col_new.append(i[:start_index] + i[start_index+7:])
  df.columns = col_new
  df.rename(columns={'유보액/093190 (2018)':'유보액/납입자본비율 (2018)', '유보액/093190 (2019)':'유보액/납입자본비율 (2019)', '유보액/093190 (2020)':'유보액/납입자본비율 (2020)'}, inplace = True)


  # 문자열 float로 변경
  for i in df.columns[3:]:
    df[i] = df[i].astype('str')
    df[i] = df[i].apply(lambda x: re.sub(",","",x))
    df[i] = df[i].astype('float')

  
  return df

  
df = make_preprocessing_fs(df)

In [10]:
(194.9 - 247.2)/247.2 * 100 

-21.156957928802584

In [11]:
df

,KIS,Stock,Name,총자산증가율 (2018),자기자본증가율 (2018),매출액증가율 (2018),총자본영업이익율 (2018),총자본순이익율 (2018),자본금순이익율 (2018),매출액순이익율 (2018),매출액영업이익율 (2018),자기자본비율 (2018),부채비율 (2018),유보액/납입자본비율 (2018),총자본회전율 (2018),자기자본회전율 (2018),당기순이익(손실) (2018),영업활동으로인한현금흐름 (2018),총자산증가율 (2019),자기자본증가율 (2019),매출액증가율 (2019),총자본영업이익율 (2019),총자본순이익율 (2019),자본금순이익율 (2019),매출액순이익율 (2019),매출액영업이익율 (2019),자기자본비율 (2019),부채비율 (2019),유보액/납입자본비율 (2019),총자본회전율 (2019),자기자본회전율 (2019),당기순이익(손실) (2019),영업활동으로인한현금흐름 (2019),총자산증가율 (2020),자기자본증가율 (2020),매출액증가율 (2020),총자본영업이익율 (2020),총자본순이익율 (2020),자본금순이익율 (2020),매출액순이익율 (2020),매출액영업이익율 (2020),자기자본비율 (2020),부채비율 (2020),유보액/납입자본비율 (2020),총자본회전율 (2020),자기자본회전율 (2020),당기순이익(손실) (2020),영업활동으로인한현금흐름 (2020)
0,036644,389500,(주)에스비비테크,46.03,-39.46,2.26,-5.23,-9.21,-168.76,-20.80,-11.80,20.22,394.44,340.07,0.44,1.39,-1.917031e+09,6.075800e+07,-21.16,-95.80,-34.70,-17.06,-21.67,-421.59,-79.56,-62.63,1.08,9183.55,-81.52,0.27,2.31,-4.789125e+09,-2.686112e+09,-23.07,747.37,9.76,-15.18,-19.01,-229.83,-49.59,-39.60,11.87,742.79,3.70,0.38,6.64,-3.276516e+09,-1.165070e+09
1,497730,296640,(주)이노룰스,81.84,215.02,-6.23,3.88,1.43,68.58,1.90,5.16,40.95,144.17,2371.70,0.75,2.16,1.770280e+08,1.148444e+09,-1.20,32.63,43.95,17.43,3.89,210.40,4.61,20.65,54.98,81.89,2891.58,0.84,1.76,6.186920e+08,4.188603e+09,-17.63,-18.42,-20.68,8.25,11.05,519.76,14.96,11.17,54.46,83.64,3253.12,0.74,1.35,1.591312e+09,1.613872e+09
2,499459,417970,모델솔루션(주),23.53,44.46,9.65,25.30,19.34,768.48,12.97,16.97,47.94,108.60,2005.45,1.49,3.33,7.210181e+09,5.469434e+09,9.91,24.51,8.93,21.58,16.81,774.92,12.00,15.41,54.31,84.14,2521.46,1.40,2.73,7.270528e+09,1.075707e+10,-6.52,-7.50,-9.16,15.68,2.97,138.47,2.36,12.49,53.74,86.09,2324.91,1.26,2.32,1.299209e+09,8.304769e+09
3,HV2035,393890,더블유씨피(주),69.90,68.28,NaN,-8.72,-7.59,-559.62,NaN,NaN,93.30,7.18,8560.18,NaN,NaN,-9.513513e+09,-8.218473e+09,172.97,31.73,NaN,-1.72,-2.21,-244.88,-18.71,-14.52,45.03,122.10,5257.56,0.12,0.20,-6.513745e+09,-2.835368e+10,21.46,-15.77,221.41,2.05,-6.54,-861.30,-27.87,8.72,31.23,220.25,4412.79,0.23,0.63,-3.117833e+10,-2.842428e+10
4,HS8995,314140,(주)알피바이오,38.75,31.69,-5.02,8.28,5.66,146.00,6.74,9.85,58.82,70.00,1558.22,0.84,1.40,4.363446e+09,6.995499e+09,29.38,7.64,7.86,3.85,3.33,105.70,4.90,5.67,48.94,104.35,1624.67,0.68,1.28,3.423335e+09,6.582410e+09,12.53,7.59,40.18,3.37,3.75,140.12,4.71,4.25,46.79,113.72,1755.41,0.79,1.66,4.615509e+09,6.454520e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1425,I73639,950110,SBI핀테크솔루션즈(주),146.31,77.94,10.83,8.05,7.49,24.84,16.69,17.92,45.68,118.91,93.99,0.45,0.88,3.963095e+09,2.682301e+09,0.36,3.69,-9.68,2.30,-0.17,-0.62,-0.59,8.09,47.20,111.87,100.17,0.28,0.61,-1.268170e+08,3.725843e+09,74.60,20.05,42.48,8.60,7.44,37.52,25.24,29.18,32.45,208.15,134.93,0.29,0.78,7.710943e+09,3.268879e+09
1426,125028,046970,(주)우리로,42.14,8.83,92.45,4.43,-0.55,-2.82,-1.06,8.50,48.09,107.95,194.50,0.52,0.96,-3.391840e+08,2.333764e+09,-0.43,-19.09,217.55,0.32,-11.97,-67.22,-8.47,0.23,39.08,155.90,126.44,1.41,3.24,-8.603656e+09,1.597561e+09,-18.92,-30.55,-4.12,-0.39,-17.91,-85.22,-11.94,-0.26,33.47,198.77,51.52,1.50,4.10,-1.162556e+10,9.402770e+08
1427,383046,113810,디젠스(주),-12.94,6.17,-29.81,-3.98,-16.02,-87.40,-31.83,-7.91,38.61,158.99,63.05,0.50,1.44,-1.185408e+10,-3.781308e+09,2.65,9.68,6.59,3.77,3.12,13.36,5.49,6.63,41.25,142.41,78.82,0.57,1.42,2.180337e+09,2.726582e+09,-10.39,-27.27,24.96,-0.54,-11.28,-46.36,-15.25,-0.73,33.48,198.66,30.06,0.74,1.97,-7.563931e+09,1.054284e+09
1428,F11411,155650,와이엠씨(주),-22.34,13.57,59.47,10.09,7.46,196.38,6.33,8.55,50.08,99.67,800.95,1.18,2.87,1.464977e+10,-8.380533e+09,-27.33,-6.80,-29.90,-3.49,-1.69,-25.00,-1.54,-3.18,64.23,55.70,746.97,1.10,1.96,-2.496629e+09,-1.223279e+10,-13.46,5.52,-10.29,8.82,6.49,75.52,5.18,7.04,78.31,27.69,831.23,1.25,1.77,7.542107e+09,1.091851e+10


In [12]:
df.isna().sum()

KIS                     0
Stock                   0
Name                    0
총자산증가율 (2018)          45
자기자본증가율 (2018)         72
매출액증가율 (2018)          63
총자본영업이익율 (2018)        30
총자본순이익율 (2018)         29
자본금순이익율 (2018)         29
매출액순이익율 (2018)         46
매출액영업이익율 (2018)        46
자기자본비율 (2018)          48
부채비율 (2018)            48
유보액/납입자본비율 (2018)      29
총자본회전율 (2018)          55
자기자본회전율 (2018)         71
당기순이익(손실) (2018)       52
영업활동으로인한현금흐름 (2018)    62
총자산증가율 (2019)          31
자기자본증가율 (2019)         66
매출액증가율 (2019)          49
총자본영업이익율 (2019)        11
총자본순이익율 (2019)         12
자본금순이익율 (2019)         10
매출액순이익율 (2019)         31
매출액영업이익율 (2019)        30
자기자본비율 (2019)          30
부채비율 (2019)            30
유보액/납입자본비율 (2019)       9
총자본회전율 (2019)          37
자기자본회전율 (2019)         65
당기순이익(손실) (2019)       33
영업활동으로인한현금흐름 (2019)    34
총자산증가율 (2020)          12
자기자본증가율 (2020)         43
매출액증가율 (2020)          32
총자본영업이익율 (2020)         0
총자본순이익율 (2020)          0
자본금순이익율 (202

# 관리종목 연도별 종목코드 뽑기

In [13]:
# 2019, 2020, 2021, 2022
df_2019_1 = pd.read_csv('/content/drive/MyDrive/핀테크 파이널/data/2019-1_수정.csv')
df_2019_2 = pd.read_csv('/content/drive/MyDrive/핀테크 파이널/data/2019-2_수정.csv')
df_2019_3 = pd.read_csv('/content/drive/MyDrive/핀테크 파이널/data/2019-3_수정.csv')

df_2020_1 = pd.read_csv('/content/drive/MyDrive/핀테크 파이널/data/2020-1_수정.csv')
df_2020_2 = pd.read_csv('/content/drive/MyDrive/핀테크 파이널/data/2020-2_수정.csv')
df_2020_3 = pd.read_csv('/content/drive/MyDrive/핀테크 파이널/data/2020-3_수정.csv')

df_2021_1 = pd.read_csv('/content/drive/MyDrive/핀테크 파이널/data/2021-1_수정.csv')
df_2021_2 = pd.read_csv('/content/drive/MyDrive/핀테크 파이널/data/2021-2_수정.csv')
df_2021_3 = pd.read_csv('/content/drive/MyDrive/핀테크 파이널/data/2021-3_수정.csv')

df_2022_1 = pd.read_csv('/content/drive/MyDrive/핀테크 파이널/data/2022-1_수정.csv')
df_2022_2 = pd.read_csv('/content/drive/MyDrive/핀테크 파이널/data/2022-2_수정.csv')

In [14]:
# 관리대상기업 데이터 연도별 concat
df_2019 = pd.concat([df_2019_1, df_2019_2, df_2019_3], axis=0).reset_index(drop=True)
df_2020 = pd.concat([df_2020_1, df_2020_2, df_2020_3], axis=0).reset_index(drop=True)
df_2021 = pd.concat([df_2021_1, df_2021_2, df_2021_3], axis=0).reset_index(drop=True)
df_2022 = pd.concat([df_2022_1, df_2022_2], axis=0).reset_index(drop=True)

In [15]:
# 관리종목 지정된 종목코드 리스트 반환하는 함수
def return_risk_code(df):
  df['종목코드'] = df['종목코드'].astype('str')
  df['종목코드'] = df['종목코드'].apply(lambda x: x.zfill(6) if len(x)!=6 else x) # 종목코드 6자리로 만들어주기

  df['지정/해제여부'] = df.apply(lambda x: 0 if '해제' in x['공시제목'] else 1, axis=1)
  df = df[df['지정/해제여부']==1].reset_index(drop=True) # 관리종목 지정인 데이터만 불러오기
  
  종목코드_list = list(df['종목코드'].unique())
  return 종목코드_list



risk_code_2019 = return_risk_code(df_2019)
risk_code_2020 = return_risk_code(df_2020)
risk_code_2021 = return_risk_code(df_2021)
risk_code_2022 = return_risk_code(df_2022)

print(f'2019년도 관리종목기업코드 개수 : {len(risk_code_2019)}')
print(f'2020년도 관리종목기업코드 개수 : {len(risk_code_2020)}')
print(f'2021년도 관리종목기업코드 개수 : {len(risk_code_2021)}')
print(f'2022년도 관리종목기업코드 개수 : {len(risk_code_2022)}')

2019년도 관리종목기업코드 개수 : 93
2020년도 관리종목기업코드 개수 : 101
2021년도 관리종목기업코드 개수 : 98
2022년도 관리종목기업코드 개수 : 73


# 기존 KisValue데이터에 관리종목여부 추가
- 2019, 2020, 2021 여부만 추가할 것.
- 모델링할 때 데이터 불균형 문제를 어느정도 해결해야할 것 같다.

In [16]:
# 일단 모든 값 0으로 초기화
# 인수합병된 기업들을 미리 지워서 그런지 위에서 출력한 개수보다 적게 나온다.

df['2019_관리종목여부'] = 0
df['2020_관리종목여부'] = 0
df['2021_관리종목여부'] = 0


df['2019_관리종목여부'] = df.apply(lambda x: 1 if x['Stock'] in risk_code_2019 else 0 , axis=1)
df['2020_관리종목여부'] = df.apply(lambda x: 1 if x['Stock'] in risk_code_2020 else 0 , axis=1)
df['2021_관리종목여부'] = df.apply(lambda x: 1 if x['Stock'] in risk_code_2021 else 0 , axis=1)

print(df['2019_관리종목여부'].value_counts())
print(df['2020_관리종목여부'].value_counts())
print(df['2021_관리종목여부'].value_counts())

0    1388
1      42
Name: 2019_관리종목여부, dtype: int64
0    1392
1      38
Name: 2020_관리종목여부, dtype: int64
0    1400
1      30
Name: 2021_관리종목여부, dtype: int64


In [ ]:
# 위에꺼 확인용 코드
# tmp=[]
# for i in risk_code_2021:
#   if i in list(df['Stock'].values):
#     tmp.append(i)
# print(len(tmp))

In [ ]:
df.describe()

,총자산증가율 (2018),자기자본증가율 (2018),매출액증가율 (2018),총자본영업이익율 (2018),총자본순이익율 (2018),자본금순이익율 (2018),매출액순이익율 (2018),매출액영업이익율 (2018),자기자본비율 (2018),부채비율 (2018),유보액/납입자본비율 (2018),총자본회전율 (2018),자기자본회전율 (2018),당기순이익(손실) (2018),영업활동으로인한현금흐름 (2018),총자산증가율 (2019),자기자본증가율 (2019),매출액증가율 (2019),총자본영업이익율 (2019),총자본순이익율 (2019),자본금순이익율 (2019),매출액순이익율 (2019),매출액영업이익율 (2019),자기자본비율 (2019),부채비율 (2019),유보액/납입자본비율 (2019),총자본회전율 (2019),자기자본회전율 (2019),당기순이익(손실) (2019),영업활동으로인한현금흐름 (2019),총자산증가율 (2020),자기자본증가율 (2020),매출액증가율 (2020),총자본영업이익율 (2020),총자본순이익율 (2020),자본금순이익율 (2020),매출액순이익율 (2020),매출액영업이익율 (2020),자기자본비율 (2020),부채비율 (2020),유보액/납입자본비율 (2020),총자본회전율 (2020),자기자본회전율 (2020),당기순이익(손실) (2020),영업활동으로인한현금흐름 (2020),2019_관리종목여부,2020_관리종목여부,2021_관리종목여부
count,1.385000e+03,1358.000000,1367.000000,1400.000000,1401.000000,1401.000000,1384.000000,1384.000000,1382.000000,1382.000000,1401.000000,1375.000000,1359.000000,1.378000e+03,1.368000e+03,1399.000000,1364.000000,1381.000000,1419.000000,1418.000000,1420.000000,1399.000000,1400.000000,1400.000000,1400.000000,1.421000e+03,1393.000000,1365.000000,1.397000e+03,1.396000e+03,1418.000000,1387.000000,1398.000000,1430.000000,1430.000000,1.430000e+03,1418.000000,1418.000000,1411.000000,1411.000000,1.430000e+03,1409.000000,1384.000000,1.407000e+03,1.407000e+03,1430.000000,1430.000000,1430.000000
mean,8.307525e+02,685.906127,21.573921,2.502879,-1.536881,70.205332,-616.313439,-471.265383,64.496852,97.685753,1697.744404,0.802269,1.610412,2.820362e+09,5.502024e+09,21.805661,32.059802,116.543526,1.872854,-2.697045,-437.646775,-369.679006,-110.732171,64.143086,99.110793,5.810453e+03,0.750718,1.476967,2.221640e+09,7.792269e+09,19.766827,56.387477,127.286381,2.678147,-1.551776,-1.107363e+03,-169.424718,-121.096855,65.231977,93.693203,4.556589e+03,0.703158,1.417753,3.842223e+09,1.026900e+10,0.029371,0.026573,0.020979
std,2.926893e+04,23315.797840,153.383154,15.853033,26.836112,2323.153378,11080.499725,10273.463568,20.955324,299.279136,9641.966500,0.633225,2.491764,1.698619e+10,1.958206e+10,77.475601,173.069248,3535.374771,16.070914,25.882170,19593.758488,6471.426055,1343.674254,20.981897,351.357481,1.616286e+05,0.573210,1.944931,2.042670e+10,2.319473e+10,83.873839,932.740963,4007.799189,14.951532,21.552174,4.287020e+04,2829.924440,1813.992693,20.137083,446.333539,1.243132e+05,0.528351,3.603946,4.631737e+10,3.016195e+10,0.168902,0.160889,0.143364
min,-6.804000e+01,-90.030000,-100.000000,-129.530000,-532.050000,-64738.350000,-353205.520000,-350051.850000,1.120000,0.910000,-8327.170000,0.010000,0.010000,-1.234088e+11,-1.672801e+11,-82.480000,-98.300000,-100.000000,-191.540000,-353.270000,-734161.410000,-184682.910000,-32961.540000,1.080000,0.430000,-1.789957e+04,0.010000,0.010000,-2.129300e+11,-1.452495e+11,-68.780000,-84.860000,-100.000000,-114.110000,-263.500000,-1.620493e+06,-90644.630000,-50203.430000,0.620000,0.370000,-5.121803e+04,0.010000,0.010000,-9.611484e+10,-6.483745e+10,0.000000,0.000000,0.000000
25%,-1.530000e+00,-1.100000,-8.145000,-0.775000,-3.400000,-31.970000,-4.920000,-0.890000,49.792500,21.345000,334.930000,0.400000,0.580000,-2.380625e+09,-1.445045e+09,-1.220000,-1.172500,-8.560000,-0.425000,-3.970000,-35.380000,-7.115000,-0.497500,49.122500,21.712500,3.364200e+02,0.390000,0.550000,-2.744422e+09,-6.504462e+08,-2.512500,-1.550000,-13.277500,-1.375000,-5.285000,-4.820500e+01,-10.525000,-2.707500,51.185000,22.160000,3.529975e+02,0.340000,0.500000,-4.359132e+09,-6.298300e+08,0.000000,0.000000,0.000000
50%,7.340000e+00,6.950000,4.380000,2.905000,2.620000,33.020000,3.670000,4.455000,65.125000,53.550000,783.650000,0.660000,1.050000,1.959088e+09,2.313819e+09,7.260000,5.500000,5.300000,2.810000,2.200000,27.930000,3.100000,4.490000,65.405000,52.885000,8.079500e+02,0.620000,1.000000,1.801617e+09,3.201173e+09,5.955000,5.790000,2.140000,2.560000,1.920000,2.608000e+01,3.020000,4.475000,67.640000,47.840000,8.398350e+02,0.590000,0.930000,1.637127e+09,4.197982e+09,0.000000,0.000000,0.000000
75%,2.496000e+01,23.480000,20.830000,8.74250

In [17]:
# 2019 관리종목대상의 2018 재무제표 평균 보기
print(df[df['2019_관리종목여부']==1]['총자산증가율 (2018)'].mean())
print(df[df['2019_관리종목여부']==1]['자기자본증가율 (2018)'].mean())
print(df[df['2019_관리종목여부']==1]['매출액증가율 (2018)'].mean())
print(df[df['2019_관리종목여부']==1]['총자본영업이익율 (2018)'].mean())
print(df[df['2019_관리종목여부']==1]['총자본순이익율 (2018)'].mean())
print(df[df['2019_관리종목여부']==1]['자본금순이익율 (2018)'].mean())
print(df[df['2019_관리종목여부']==1]['매출액순이익율 (2018)'].mean())
print(df[df['2019_관리종목여부']==1]['매출액영업이익율 (2018)'].mean())
print(df[df['2019_관리종목여부']==1]['자기자본비율 (2018)'].mean())
print(df[df['2019_관리종목여부']==1]['부채비율 (2018)'].mean())
print(df[df['2019_관리종목여부']==1]['유보액/납입자본비율 (2018)'].mean())
print(df[df['2019_관리종목여부']==1]['총자본회전율 (2018)'].mean())
print(df[df['2019_관리종목여부']==1]['자기자본회전율 (2018)'].mean())
print(df[df['2019_관리종목여부']==1]['당기순이익(손실) (2018)'].mean())
print(df[df['2019_관리종목여부']==1]['영업활동으로인한현금흐름 (2018)'].mean())

8.886904761904763
7.115714285714287
-3.3880952380952385
-8.333809523809524
-17.688333333333333
-106.68761904761905
-51.57119047619048
-26.022857142857145
49.95333333333333
142.62261904761905
564.6380952380953
0.5435714285714286
1.2719047619047619
-10126616951.219513
-1637915292.682927


In [18]:
# 위랑 비교해서 2019 관리종목대상의 2018 재무제표 평균 보기
print(df[df['2019_관리종목여부']==0]['총자산증가율 (2018)'].mean())
print(df[df['2019_관리종목여부']==0]['자기자본증가율 (2018)'].mean())
print(df[df['2019_관리종목여부']==0]['매출액증가율 (2018)'].mean())
print(df[df['2019_관리종목여부']==0]['총자본영업이익율 (2018)'].mean())
print(df[df['2019_관리종목여부']==0]['총자본순이익율 (2018)'].mean())
print(df[df['2019_관리종목여부']==0]['자본금순이익율 (2018)'].mean())
print(df[df['2019_관리종목여부']==0]['매출액순이익율 (2018)'].mean()) # 이거 아웃라이어 있는거같은데 살짝 지우자. 
print(df[df['2019_관리종목여부']==0]['매출액영업이익율 (2018)'].mean()) # 이것도 아웃라이어 데이터 삭제해주자.
print(df[df['2019_관리종목여부']==0]['자기자본비율 (2018)'].mean())

print(df[df['2019_관리종목여부']==0]['부채비율 (2018)'].mean())
print(df[df['2019_관리종목여부']==0]['자기자본비율 (2018)'].mean())
print(df[df['2019_관리종목여부']==0]['유보액/납입자본비율 (2018)'].mean())
print(df[df['2019_관리종목여부']==0]['총자본회전율 (2018)'].mean())
print(df[df['2019_관리종목여부']==0]['자기자본회전율 (2018)'].mean())
print(df[df['2019_관리종목여부']==0]['영업활동으로인한현금흐름 (2018)'].mean())

856.4549218168281
707.5696504559271
22.365169811320758
2.838033873343152
-1.037718910963944
75.67222222222223
-633.9879359165425
-485.19994783904616
64.95269402985075
96.27728358208955
64.95269402985075
1732.7631420161883
0.8104201050262565
1.6212072892938498
5722624618.688771


2019년만 슬쩍 봤는데 평균으로 찍어봐도 유의미한 결과가 나온다.


## 일단 2018년 컬럼 모두가 null이면 drop(axis=0) 하자
- 2017년이나 2018년 이전부터 설립된 기업만을 대상으로 하기 위해
- 내가 확인한결과 2020, 2021은 모두 nan인게 없다.

In [19]:
df3 = df.iloc[:,3:18].T
all_nan_2018 = list(df3.isna().sum()[df3.isna().sum().values==15].index)

df = df.drop(all_nan_2018, axis=0).reset_index(drop=True)

In [20]:
print(df['2019_관리종목여부'].value_counts())
print(df['2020_관리종목여부'].value_counts())
print(df['2021_관리종목여부'].value_counts())

0    1359
1      42
Name: 2019_관리종목여부, dtype: int64
0    1363
1      38
Name: 2020_관리종목여부, dtype: int64
0    1372
1      29
Name: 2021_관리종목여부, dtype: int64


In [21]:
df.isna().sum()

KIS                     0
Stock                   0
Name                    0
총자산증가율 (2018)          16
자기자본증가율 (2018)         43
매출액증가율 (2018)          34
총자본영업이익율 (2018)         1
총자본순이익율 (2018)          0
자본금순이익율 (2018)          0
매출액순이익율 (2018)         17
매출액영업이익율 (2018)        17
자기자본비율 (2018)          19
부채비율 (2018)            19
유보액/납입자본비율 (2018)       0
총자본회전율 (2018)          26
자기자본회전율 (2018)         42
당기순이익(손실) (2018)       23
영업활동으로인한현금흐름 (2018)    33
총자산증가율 (2019)           2
자기자본증가율 (2019)         37
매출액증가율 (2019)          20
총자본영업이익율 (2019)         1
총자본순이익율 (2019)          2
자본금순이익율 (2019)          0
매출액순이익율 (2019)         13
매출액영업이익율 (2019)        12
자기자본비율 (2019)          21
부채비율 (2019)            21
유보액/납입자본비율 (2019)       0
총자본회전율 (2019)          19
자기자본회전율 (2019)         47
당기순이익(손실) (2019)       23
영업활동으로인한현금흐름 (2019)    24
총자산증가율 (2020)           3
자기자본증가율 (2020)         32
매출액증가율 (2020)          14
총자본영업이익율 (2020)         0
총자본순이익율 (2020)          0
자본금순이익율 (202

# 네이버 파이낸스로 채울수 있는부분 채워보기
- 우리는 연도를 기준으로 ??

<pre>
- 총자산증가율 : (올해 총자산 - 전년 총자산) / 전년 총자산 * 100
- 자기자본증가율 : (올해 자기자본 - 전년 자기자본)/전년 자기자본 * 100 
- 매출액증가율 : (올해 매출액 - 전년도 매출액) / 전년도 매출액 * 100 
- 총자본영업이익율 : (영업이익/총자산) * 100
- 총자본순이익율 : ROA랑 같다. (당기순이익/총자산) * 100
- 매출액순이익율 : ROS랑 같다. (당기순이익) / (매출액) * 100
- 매출액영업이익율 : 영업이익 / 매출액 * 100
- 자기자본비율 : BIS랑 같다. 자기자본 / 총자산 * 100
- 부채비율 : (부채총계 / 자본총계) * 100
- 총자본회전율 : (매출액 / 총자본 (== 총자산) ) * 100
- 자기자본회전율 : (매출액 / 자기자본 ) * 100
- 당기순이익(손실) : 이거 그냥 당기순이익으로 메꿔도 되나??
- 영업활동으로인한현금흐름 : CFO랑 같다. 영업이익 + 감가상각비 - 법인세


필요 재무제표 : 총자산, 자기자본, 매출액, 

In [22]:
df.head(30)

,KIS,Stock,Name,총자산증가율 (2018),자기자본증가율 (2018),매출액증가율 (2018),총자본영업이익율 (2018),총자본순이익율 (2018),자본금순이익율 (2018),매출액순이익율 (2018),매출액영업이익율 (2018),자기자본비율 (2018),부채비율 (2018),유보액/납입자본비율 (2018),총자본회전율 (2018),자기자본회전율 (2018),당기순이익(손실) (2018),영업활동으로인한현금흐름 (2018),총자산증가율 (2019),자기자본증가율 (2019),매출액증가율 (2019),총자본영업이익율 (2019),총자본순이익율 (2019),자본금순이익율 (2019),매출액순이익율 (2019),매출액영업이익율 (2019),자기자본비율 (2019),부채비율 (2019),유보액/납입자본비율 (2019),총자본회전율 (2019),자기자본회전율 (2019),당기순이익(손실) (2019),영업활동으로인한현금흐름 (2019),총자산증가율 (2020),자기자본증가율 (2020),매출액증가율 (2020),총자본영업이익율 (2020),총자본순이익율 (2020),자본금순이익율 (2020),매출액순이익율 (2020),매출액영업이익율 (2020),자기자본비율 (2020),부채비율 (2020),유보액/납입자본비율 (2020),총자본회전율 (2020),자기자본회전율 (2020),당기순이익(손실) (2020),영업활동으로인한현금흐름 (2020),2019_관리종목여부,2020_관리종목여부,2021_관리종목여부
0,036644,389500,(주)에스비비테크,46.03,-39.46,2.26,-5.23,-9.21,-168.76,-20.80,-11.80,20.22,394.44,340.07,0.44,1.39,-1.917031e+09,6.075800e+07,-21.16,-95.80,-34.70,-17.06,-21.67,-421.59,-79.56,-62.63,1.08,9183.55,-81.52,0.27,2.31,-4.789125e+09,-2.686112e+09,-23.07,747.37,9.76,-15.18,-19.01,-229.83,-49.59,-39.60,11.87,742.79,3.70,0.38,6.64,-3.276516e+09,-1.165070e+09,0,0,0
1,497730,296640,(주)이노룰스,81.84,215.02,-6.23,3.88,1.43,68.58,1.90,5.16,40.95,144.17,2371.70,0.75,2.16,1.770280e+08,1.148444e+09,-1.20,32.63,43.95,17.43,3.89,210.40,4.61,20.65,54.98,81.89,2891.58,0.84,1.76,6.186920e+08,4.188603e+09,-17.63,-18.42,-20.68,8.25,11.05,519.76,14.96,11.17,54.46,83.64,3253.12,0.74,1.35,1.591312e+09,1.613872e+09,0,0,0
2,499459,417970,모델솔루션(주),23.53,44.46,9.65,25.30,19.34,768.48,12.97,16.97,47.94,108.60,2005.45,1.49,3.33,7.210181e+09,5.469434e+09,9.91,24.51,8.93,21.58,16.81,774.92,12.00,15.41,54.31,84.14,2521.46,1.40,2.73,7.270528e+09,1.075707e+10,-6.52,-7.50,-9.16,15.68,2.97,138.47,2.36,12.49,53.74,86.09,2324.91,1.26,2.32,1.299209e+09,8.304769e+09,0,0,0
3,HV2035,393890,더블유씨피(주),69.90,68.28,NaN,-8.72,-7.59,-559.62,NaN,NaN,93.30,7.18,8560.18,NaN,NaN,-9.513513e+09,-8.218473e+09,172.97,31.73,NaN,-1.72,-2.21,-244.88,-18.71,-14.52,45.03,122.10,5257.56,0.12,0.20,-6.513745e+09,-2.835368e+10,21.46,-15.77,221.41,2.05,-6.54,-861.30,-27.87,8.72,31.23,220.25,4412.79,0.23,0.63,-3.117833e+10,-2.842428e+10,0,0,0
4,HS8995,314140,(주)알피바이오,38.75,31.69,-5.02,8.28,5.66,146.00,6.74,9.85,58.82,70.00,1558.22,0.84,1.40,4.363446e+09,6.995499e+09,29.38,7.64,7.86,3.85,3.33,105.70,4.90,5.67,48.94,104.35,1624.67,0.68,1.28,3.423335e+09,6.582410e+09,12.53,7.59,40.18,3.37,3.75,140.12,4.71,4.25,46.79,113.72,1755.41,0.79,1.66,4.615509e+09,6.454520e+09,0,0,0
5,389238,368970,(주)오에스피,NaN,NaN,NaN,21.66,15.28,284.44,9.37,13.29,54.28,84.25,910.65,1.63,3.00,1.137761e+09,NaN,6.86,28.37,22.75,37.91,29.04,559.35,15.01,19.60,65.20,53.37,1197.33,1.93,3.23,2.237419e+09,9.420240e+08,98.27,131.04,4.45,33.15,27.91,171.72,21.28,25.28,75.98,31.62,246.63,1.31,1.81,3.313228e+09,4.079556e+09,0,0,0
6,IW9267,394280,오픈엣지테크놀로지(주),24565.93,24767.99,NaN,26.41,25.40,2683.41,36.49,37.94,93.82,6.59,17064.75,0.70,0.74,4.735680e+08,NaN,173.72,110.40,-4.58,-41.29,-39.81,-12516.99,-223.03,-231.34,72.12,38.67,30650.58,0.18,0.23,-2.761999e+09,-2.927803e+09,-10.72,-61.93,-12.09,-78.12,-78.38,-30718.01,-692.55,-690.26,30.76,225.14,10951.73,0.11,0.22,-7.539429e+09,-6.780314e+09,0,0,0
7,484917,148930,(주)에이치와이티씨,6.46,NaN,48.83,21.61,14.00,74.47,10.01,15.46,NaN,NaN,-190.55,1.40,NaN,1.940563e+09,4.664951e+09,14.33,NaN,2.59,17.25,15.36,73.95,11.84,13.29,33.88,195.13,47.25,1.30,NaN,2.354471e+09,3.519516e+09,31.35,71.09,7.13,20.22,17.24,86.68,15.31,17.95,44.13,126.58,151.92,1.13,2.84,3.261039e+09,5.717473e+09,0,0,0
8,894281,107600,(주)새빗켐,24.53,15.87,5.81,14.03,10.14,51.67,7.86,10.88,54.11,84.81,205.99,1.29,2.31,1.058424e+09,NaN,5.48,13.30,15.49,13.12,7.74,44.98,5.93,10.05,58.12,72.06,246.68,1.31,2.33,9.213920e+08,2.448289e+09,24.85,25.15,34.88,15.83,13.64,91.48,8.94,10.37,58.26,71.64,333.87,1.53,2.62,1.873926e+09,3.736398e+09,0,0,0
9,867055,129920,(주)대성하이텍,8.36,26.30,7.67,3.96,1.57,162.88,2.10,5.32,38.03,162.97,4012.35,0.74,2.10,1.865670e+09,8.755060e+08,2.05,9.07,-17.48,4.13,2.48,254.75,4.24,7.07

# 네이버 파이낸스에 없는 데이터는 kisvalue에도 없는 것 같다.


# OpenDartAPI 사용해서 그나마 있는거 채우기
- opendartApi에는 연결재무제표 or 재무제표밖에 없다.

In [ ]:
# 일단 모두 연간으로 계산해야할 것 같다.

# 변화율을 구해야하므로 2017년부터 2020년 재무제표 데이터까지 구하고 비율 계산
# 총자산증가율

In [23]:
import OpenDartReader

### 0. 객체 생성 ###
# 객체 생성 (API KEY 지정) 
api_key = 'fed627c93ccbd6bbc2ad3c3769f4c41be9e377eb'

dart = OpenDartReader(api_key) 

result = dart.finstate('LG전자', 2020, '11011') # 2020년 사업보고서 내 재무 정보 가져오기
result = result.loc[result['fs_nm']=='연결재무제표'] # 연결재무제표 필터링

# account_nm이 당기순이익, 영업이익, 매출액 중 하나의 경우에만 가져오기
result = result.loc[result['account_nm'].isin(['당기순이익', '영업이익', '영업이익', '매출액'])]

# 열 필터링
result = result.loc[:,['account_nm', 'fs_nm', 'sj_nm', 'thstrm_dt', 'thstrm_amount']]
display(result)

,account_nm,fs_nm,sj_nm,thstrm_dt,thstrm_amount
9,매출액,연결재무제표,손익계산서,2020.01.01 ~ 2020.12.31,"63,262,046,000,000"
10,영업이익,연결재무제표,손익계산서,2020.01.01 ~ 2020.12.31,"3,194,987,000,000"
12,당기순이익,연결재무제표,손익계산서,2020.01.01 ~ 2020.12.31,"2,063,790,000,000"


# 필요한 재무제표 데이터 불러오기
-  개별은 아래에서 재무제표만 뽑아내면 된다.
- indicator 다시 적용(추가로 구할수있는거 있으면 구하게)
- 11011(사업보고서) 만 갖고올수있게끔

In [24]:
# 기업 연도 한개씩으로 내가 새로 만들어보기
# 정민이한테 물어보기
stock_code = '018700'
year = 2017
indicators = ['자산총계', '부채총계', '자본총계', '매출액', '영업이익', '당기순이익', '법인세차감전 순이익']


result = dart.finstate(stock_code, year,'11011') # 사업보고서 데이터 가져오기
# if report is None: # 리포트가 없다면 (참고 : 리포트가 없으면 None을 반환함)
result = result.loc[result['fs_nm']=='재무제표'] # 연결재무제표 아닌 재무제표 필터링
result = result.loc[result['account_nm'].isin(indicators)]
data = [] 
for y, c in zip([year], ['thstrm_amount']):
  record = [stock_code, y]
  for indic in indicators:
    # account_nm이 indic인 행의 c 컬럼 값을 가져옴
    if sum(result['account_nm']==indic)>0: # 해당 지표가 있다면 추가(지표가 없는 경우가 있음)
      value = result.loc[result['account_nm']==indic, c].iloc[0]
    else:
      value = np.nan

    record.append(value)

  data.append(record)
tt= pd.DataFrame(data, columns = ['기업', '연도'] + indicators)  
tt

,기업,연도,자산총계,부채총계,자본총계,매출액,영업이익,당기순이익,법인세차감전 순이익
0,018700,2017,"40,475,076,097","2,607,763,705","37,867,312,392","3,499,939,883","-604,655,894","-902,076,584","-112,999,374"


In [25]:
# 함수화


def crawl_finance(stock_code, year, indicators):


  result = dart.finstate(stock_code, year,'11011') # 사업보고서 데이터 가져오기
  # if report is None: # 리포트가 없다면 (참고 : 리포트가 없으면 None을 반환함)
  result = result.loc[result['fs_nm']=='재무제표'] # 연결재무제표 아닌 재무제표 필터링
  result = result.loc[result['account_nm'].isin(indicators)]
  data = [] 
  for y, c in zip([year], ['thstrm_amount']):
    record = [stock_code, y]
    for indic in indicators:
      # account_nm이 indic인 행의 c 컬럼 값을 가져옴
      if sum(result['account_nm']==indic)>0: # 해당 지표가 있다면 추가(지표가 없는 경우가 있음)
        value = result.loc[result['account_nm']==indic, c].iloc[0]
      else:
        value = np.nan

      record.append(value)

    data.append(record)
  tt= pd.DataFrame(data, columns = ['기업', '연도'] + indicators)  
  tt
  return tt

In [26]:
year = [2017, 2018, 2019, 2020]
indicators = ['자산총계', '부채총계', '자본총계', '매출액', '영업이익', '당기순이익', '법인세차감전 순이익']

# 널값인 데이터만 보기
df2 = df.copy()
df2['NaN_cnt'] = df2.isnull().sum(1)

# 재무제표 crawling이 필요한 코드
stock_code = list(df2[df2['NaN_cnt']>0]['Stock'].values)
 
# 널값이 있는 종목코드 재무제표만 저장하는 데이터프레임
null_fill_data = pd.DataFrame(columns = ['기업', '연도'] + indicators) 
for i in stock_code:
  for j in year:
     try:
       tt = crawl_finance(i, j, indicators)
       null_fill_data = pd.concat([null_fill_data, tt], axis=0)
     except:
       pass

null_fill_data

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없

,기업,연도,자산총계,부채총계,자본총계,매출액,영업이익,당기순이익,법인세차감전 순이익
0,347700,2020,"9,539,670,072","7,353,748,337","2,185,921,735","2,665,429,178","-3,657,769,678","-9,308,741,731","-9,308,741,731"
0,338220,2020,"9,587,094,279","13,229,235,096","-3,642,140,817","1,257,218,734","-9,721,688,063","-11,108,598,035","-11,108,598,035"
0,334970,2020,"50,232,970,362","51,630,407,000","-1,397,436,638",-,"-22,601,457,755","-31,033,281,007","-31,033,281,007"
0,340930,2020,"40,790,146,981","20,485,252,169","20,304,894,812","58,978,131,009","7,718,043,746","4,765,270,114","5,240,984,389"
0,321820,2020,"6,027,806,557","7,157,246,083","-1,129,439,526","28,799,193,142","-3,232,982,943","-3,242,623,376","-3,242,623,376"
...,...,...,...,...,...,...,...,...,...
0,291230,2019,"15,055,376,421","919,220,457","14,136,155,964",-,"-26,942,340","164,510,142","182,705,299"
0,291230,2020,"15,152,725,341","945,135,877","14,207,589,464",NaN,"-48,677,430","71,433,500","72,983,568"
0,284620,2018,"10,599,449,914","772,060,139","9,827,389,775",0,"-25,917,490","38,946,647","49,931,598"
0,284620,2019,"10,697,358,214","789,886,882","9,907,471,332",0,"-44,177,300","80,081,557","90,492,514"


In [27]:
# 중복된 행 제거
null_fill_data.drop_duplicates(inplace = True)

In [28]:
# 숫자로 모두 변환
def str_to_float(value):
  if type(value)==float: # nan의 경우에는 문자가 아니라, 이미 float 취급됨
    return value
  elif value =='-': # -로 되어 있으면 0으로 변환
    return 0
  else:
    return float(value.replace(',',''))

for indc in indicators:
  null_fill_data[indc] = null_fill_data[indc].apply(str_to_float)

In [29]:
# 기업, 연도 기준으로 sorting
null_fill_data.sort_values(by=['기업', '연도'], inplace= True)
null_fill_data = null_fill_data.reset_index(drop=True)

In [30]:
null_fill_data.head(30)

,기업,연도,자산총계,부채총계,자본총계,매출액,영업이익,당기순이익,법인세차감전 순이익
0,006730,2017,1.502578e+12,1.018991e+12,4.835862e+11,5.754273e+10,-1.400903e+10,-2.655790e+10,-2.805165e+10
1,006730,2018,1.724660e+12,1.076229e+12,6.484311e+11,1.081047e+11,-1.541956e+07,-1.037741e+10,-6.509073e+09
2,006730,2019,1.811212e+12,1.138809e+12,6.724037e+11,1.279366e+11,1.443905e+10,4.709363e+07,2.180952e+10
3,006730,2020,1.818179e+12,1.170517e+12,6.476618e+11,8.410463e+10,-6.472848e+09,-1.682075e+10,-1.777866e+10
4,018700,2017,4.047508e+10,2.607764e+09,3.786731e+10,3.499940e+09,-6.046559e+08,-9.020766e+08,-1.129994e+08
5,018700,2018,3.850006e+10,2.637741e+09,3.586232e+10,8.732299e+09,-2.019640e+09,-5.642155e+09,-5.642155e+09
6,018700,2019,3.715530e+10,3.033724e+09,3.412158e+10,3.506172e+09,-3.320440e+09,-7.076383e+09,-7.076383e+09
7,018700,2020,5.118833e+10,1.034894e+10,4.083939e+10,2.464416e+10,9.113775e+08,-8.545291e+09,-8.545291e+09
8,019570,2018,1.702666e+10,2.486696e+09,1.453996e+10,NaN,-7.409675e+08,-1.060834e+09,-1.060834e+09
9,019570,2019,2.594744e+10,5.790251e+09,2.015719e+10,NaN,-3.339529e+09,-3.261240e+09,-3.261240e+09


In [31]:
null_fill_data['자산총계'].diff(1)

0               NaN
1      2.220827e+11
2      8.655190e+10
3      6.966514e+09
4     -1.777704e+12
           ...     
151   -1.109239e+10
152   -3.125048e+10
153    4.613555e+10
154   -6.281865e+09
155   -4.934685e+10
Name: 자산총계, Length: 156, dtype: float64

In [32]:
null_fill_data['매출액'].shift(1)

0               NaN
1      5.754273e+10
2      1.081047e+11
3      1.279366e+11
4      8.410463e+10
           ...     
151    4.525714e+10
152    5.897813e+10
153    2.665429e+09
154    4.591485e+09
155    2.479277e+09
Name: 매출액, Length: 156, dtype: float64

In [33]:
null_fill_data

,기업,연도,자산총계,부채총계,자본총계,매출액,영업이익,당기순이익,법인세차감전 순이익
0,006730,2017,1.502578e+12,1.018991e+12,4.835862e+11,5.754273e+10,-1.400903e+10,-2.655790e+10,-2.805165e+10
1,006730,2018,1.724660e+12,1.076229e+12,6.484311e+11,1.081047e+11,-1.541956e+07,-1.037741e+10,-6.509073e+09
2,006730,2019,1.811212e+12,1.138809e+12,6.724037e+11,1.279366e+11,1.443905e+10,4.709363e+07,2.180952e+10
3,006730,2020,1.818179e+12,1.170517e+12,6.476618e+11,8.410463e+10,-6.472848e+09,-1.682075e+10,-1.777866e+10
4,018700,2017,4.047508e+10,2.607764e+09,3.786731e+10,3.499940e+09,-6.046559e+08,-9.020766e+08,-1.129994e+08
...,...,...,...,...,...,...,...,...,...
151,340930,2020,4.079015e+10,2.048525e+10,2.030489e+10,5.897813e+10,7.718044e+09,4.765270e+09,5.240984e+09
152,347700,2020,9.539670e+09,7.353748e+09,2.185922e+09,2.665429e+09,-3.657770e+09,-9.308742e+09,-9.308742e+09
153,348150,2020,5.567522e+10,1.541278e+09,5.413395e+10,4.591485e+09,-1.021386e+10,-5.631520e+10,-5.631520e+10
154,354200,2020,4.939336e+10,7.944209e+09,4.144915e+10,2.479277e+09,-6.104003e+09,-8.227767e+09,-8.227767e+09


In [34]:
# 총자산증가율, 자기자본증가율, 매출액증가율, 총자본영업이익율, 총자본순이익율, 매출액순이익율, 매출액영업이익율, 자기자본비율, 부채비율, 총자본회전율, 자기자본회전율, 당기순이익, 영업활동으로인한현금흐름
def make_재무제표_파생변수(df):
  # 기업코드 unique 가져오기
  unique_기업코드 = list(df['기업'].unique())

  # 총자산증가율=[]
  # 자기자본증가율=[]
  # 매출액증가율=[]
  # 총자본영업이익율=[]
  # 총자본순이익율=[]
  # 매출액순이익율=[]
  # 매출액영업이익율=[]
  # 자기자본비율=[]
  # 부채비율=[]
  # 총자본회전율=[]
  # 자기자본회전율=[]
  # 당기순이익=[]
  # 영업활동으로인한현금흐름=[]

  tmp_total = pd.DataFrame()

  for i in unique_기업코드:
    
    tmp = df[df['기업']==i]
    
    # 총자산증가율 - 수정완료
    tmp['총자산증가율'] = tmp['자산총계'].diff(1) / tmp['자산총계'].shift(1) * 100

    # 자기자본증가율 - 수정완료
    tmp['자기자본증가율'] = (tmp['자본총계'] - tmp['자본총계'].shift(1))/tmp['자본총계'] * 100

    # 매출액증가율 - 수정완료
    tmp['매출액증가율'] = (tmp['매출액'] - tmp['매출액'].shift(1)) / tmp['매출액'].shift(1) * 100 

    # 총자본영업이익율 - 얼추비슷한 비율로 나온다. -> 다르다
    tmp['총자본영업이익율'] = (tmp['영업이익'] / tmp['자산총계']) * 100

    # 총자본순이익율 - 얼추비슷한 비율로 나온다. -> 다르다
    tmp['총자본순이익율'] = (tmp['당기순이익'] / tmp['자산총계']) * 100

    # 매출액순이익율 - 수정완료
    tmp['매출액순이익율'] = (tmp['당기순이익']/tmp['매출액'])*100

    # 매출액영업이익율 - 수정완료
    tmp['매출액영업이익율'] = (tmp['영업이익'] / tmp['매출액']) * 100

    # 자기자본비율 - 수정완료
    tmp['자기자본비율'] = (tmp['자본총계'] / tmp['자산총계']) * 100

    # 부채비율 - 수정완료
    tmp['부채비율'] = (tmp['부채총계'] / tmp['자본총계']) * 100

    # 총자본회전율 - 이건값이 안맞게 나옴
    tmp['총자본회전율'] = (tmp['매출액'] / tmp['자산총계']) * 100

    # 자기자본회전율 - 이것도 값이 안맞게 나온다.
    tmp['자기자본회전율'] = (tmp['매출액'] / tmp['자본총계']) * 100

    # 당기순이익(손실)
    tmp['당기순이익(손실)'] = (tmp['당기순이익'])

    # array형태인데 shape 한번봐야겠다. 괄호 안에 있는지 없는지
    # 총자산증가율.append(tmp['총자산증가율'].values)
    # 자기자본증가율.append(tmp['자기자본증가율'].values)
    # 매출액증가율.append(tmp['매출액증가율'].values)
    # 총자본영업이익율.append(tmp['총자본영업이익율'].values)
    # 총자본순이익율.append(tmp['총자본순이익율'].values)
    # 매출액순이익율.append(tmp['매출액순이익율'].values)
    # 매출액영업이익율.append(tmp['매출액영업이익율'].values)
    # 자기자본비율.append(tmp['자기자본비율'].values)
    # 부채비율.append(tmp['부채비율'].values)
    # 총자본회전율.append(tmp['총자본회전율'].values)
    # 자기자본회전율.append(tmp['자기자본회전율'].values)
    

    tmp = tmp[['기업', '연도', '총자산증가율', '자기자본증가율','매출액증가율','총자본영업이익율','총자본순이익율','매출액순이익율','매출액영업이익율', '자기자본비율', '부채비율', '총자본회전율', '자기자본회전율', '당기순이익(손실)']]
    tmp_total = pd.concat([tmp_total, tmp], axis=0)
  # df['자산총계'].diff(1)

  return tmp_total



tmp_total = make_재무제표_파생변수(null_fill_data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [35]:
tmp_total.head(30)

,기업,연도,총자산증가율,자기자본증가율,매출액증가율,총자본영업이익율,총자본순이익율,매출액순이익율,매출액영업이익율,자기자본비율,부채비율,총자본회전율,자기자본회전율,당기순이익(손실)
0,006730,2017,NaN,NaN,NaN,-0.932333,-1.767489,-46.153355,-24.345440,32.183772,210.715598,3.829601,11.899169,-2.655790e+10
1,006730,2018,14.780118,25.422116,87.868600,-0.000894,-0.601708,-9.599407,-0.014264,37.597608,165.974364,6.268175,16.671738,-1.037741e+10
2,006730,2019,5.018489,3.565213,18.345080,0.797204,0.002600,0.036810,11.286099,37.124510,169.363825,7.063591,19.026759,4.709363e+07
3,006730,2020,0.384633,-3.820188,-34.260713,-0.356007,-0.925143,-19.999794,-7.696186,35.621458,180.729669,4.625762,12.985887,-1.682075e+10
4,018700,2017,NaN,NaN,NaN,-1.493897,-2.228721,-25.774059,-17.276179,93.557112,6.886582,8.647148,9.242641,-9.020766e+08
5,018700,2018,-4.879579,-5.590801,149.498529,-5.245811,-14.654924,-64.612484,-23.128394,93.148735,7.355189,22.681258,24.349507,-5.642155e+09
6,018700,2019,-3.492879,-5.101591,-59.848235,-8.936651,-19.045420,-201.826478,-94.702701,91.835017,8.890926,9.436532,10.275527,-7.076383e+09
7,018700,2020,37.768565,16.449343,602.879489,1.780440,-16.693827,-34.674703,3.698148,79.782622,25.340579,48.144109,60.344105,-8.545291e+09
8,019570,2018,NaN,NaN,NaN,-4.351808,-6.230429,NaN,NaN,85.395283,17.102487,NaN,NaN,-1.060834e+09
9,019570,2019,52.392984,27.867087,NaN,-12.870363,-12.568642,NaN,NaN,77.684690,28.725492,NaN,NaN,-3.261240e+09


In [36]:
tmp_total[tmp_total['기업']=='284620']

,기업,연도,총자산증가율,자기자본증가율,매출액증가율,총자본영업이익율,총자본순이익율,매출액순이익율,매출액영업이익율,자기자본비율,부채비율,총자본회전율,자기자본회전율,당기순이익(손실)
102,284620,2018,NaN,NaN,NaN,-0.244517,0.367440,inf,-inf,92.716036,7.856208,0.0,0.0,3.894665e+07
103,284620,2019,0.923711,0.808295,NaN,-0.412974,0.748611,inf,-inf,92.616057,7.972639,0.0,0.0,8.008156e+07
104,284620,2020,188.394278,19.794756,NaN,-27.284750,-43.242810,-inf,-inf,40.040260,149.748631,0.0,0.0,-1.334065e+10


# 기존 df와 merge작업
- left join 이후 np.where로 null값 채우기

In [37]:
# 2017년이 아닌 연도만 기준으로(2017년 데이터는 변화율 구하기 위해서 사용한것임.)
tmp_total = tmp_total[tmp_total['연도']!=2017].reset_index(drop=True)

# 일단 inf나온것은 nan으로 모두 대체한다.
tmp_total.replace(np.inf, np.nan, inplace = True)
tmp_total.replace(-np.inf, np.nan, inplace = True)

In [38]:
df

,KIS,Stock,Name,총자산증가율 (2018),자기자본증가율 (2018),매출액증가율 (2018),총자본영업이익율 (2018),총자본순이익율 (2018),자본금순이익율 (2018),매출액순이익율 (2018),매출액영업이익율 (2018),자기자본비율 (2018),부채비율 (2018),유보액/납입자본비율 (2018),총자본회전율 (2018),자기자본회전율 (2018),당기순이익(손실) (2018),영업활동으로인한현금흐름 (2018),총자산증가율 (2019),자기자본증가율 (2019),매출액증가율 (2019),총자본영업이익율 (2019),총자본순이익율 (2019),자본금순이익율 (2019),매출액순이익율 (2019),매출액영업이익율 (2019),자기자본비율 (2019),부채비율 (2019),유보액/납입자본비율 (2019),총자본회전율 (2019),자기자본회전율 (2019),당기순이익(손실) (2019),영업활동으로인한현금흐름 (2019),총자산증가율 (2020),자기자본증가율 (2020),매출액증가율 (2020),총자본영업이익율 (2020),총자본순이익율 (2020),자본금순이익율 (2020),매출액순이익율 (2020),매출액영업이익율 (2020),자기자본비율 (2020),부채비율 (2020),유보액/납입자본비율 (2020),총자본회전율 (2020),자기자본회전율 (2020),당기순이익(손실) (2020),영업활동으로인한현금흐름 (2020),2019_관리종목여부,2020_관리종목여부,2021_관리종목여부
0,036644,389500,(주)에스비비테크,46.03,-39.46,2.26,-5.23,-9.21,-168.76,-20.80,-11.80,20.22,394.44,340.07,0.44,1.39,-1.917031e+09,6.075800e+07,-21.16,-95.80,-34.70,-17.06,-21.67,-421.59,-79.56,-62.63,1.08,9183.55,-81.52,0.27,2.31,-4.789125e+09,-2.686112e+09,-23.07,747.37,9.76,-15.18,-19.01,-229.83,-49.59,-39.60,11.87,742.79,3.70,0.38,6.64,-3.276516e+09,-1.165070e+09,0,0,0
1,497730,296640,(주)이노룰스,81.84,215.02,-6.23,3.88,1.43,68.58,1.90,5.16,40.95,144.17,2371.70,0.75,2.16,1.770280e+08,1.148444e+09,-1.20,32.63,43.95,17.43,3.89,210.40,4.61,20.65,54.98,81.89,2891.58,0.84,1.76,6.186920e+08,4.188603e+09,-17.63,-18.42,-20.68,8.25,11.05,519.76,14.96,11.17,54.46,83.64,3253.12,0.74,1.35,1.591312e+09,1.613872e+09,0,0,0
2,499459,417970,모델솔루션(주),23.53,44.46,9.65,25.30,19.34,768.48,12.97,16.97,47.94,108.60,2005.45,1.49,3.33,7.210181e+09,5.469434e+09,9.91,24.51,8.93,21.58,16.81,774.92,12.00,15.41,54.31,84.14,2521.46,1.40,2.73,7.270528e+09,1.075707e+10,-6.52,-7.50,-9.16,15.68,2.97,138.47,2.36,12.49,53.74,86.09,2324.91,1.26,2.32,1.299209e+09,8.304769e+09,0,0,0
3,HV2035,393890,더블유씨피(주),69.90,68.28,NaN,-8.72,-7.59,-559.62,NaN,NaN,93.30,7.18,8560.18,NaN,NaN,-9.513513e+09,-8.218473e+09,172.97,31.73,NaN,-1.72,-2.21,-244.88,-18.71,-14.52,45.03,122.10,5257.56,0.12,0.20,-6.513745e+09,-2.835368e+10,21.46,-15.77,221.41,2.05,-6.54,-861.30,-27.87,8.72,31.23,220.25,4412.79,0.23,0.63,-3.117833e+10,-2.842428e+10,0,0,0
4,HS8995,314140,(주)알피바이오,38.75,31.69,-5.02,8.28,5.66,146.00,6.74,9.85,58.82,70.00,1558.22,0.84,1.40,4.363446e+09,6.995499e+09,29.38,7.64,7.86,3.85,3.33,105.70,4.90,5.67,48.94,104.35,1624.67,0.68,1.28,3.423335e+09,6.582410e+09,12.53,7.59,40.18,3.37,3.75,140.12,4.71,4.25,46.79,113.72,1755.41,0.79,1.66,4.615509e+09,6.454520e+09,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1396,I73639,950110,SBI핀테크솔루션즈(주),146.31,77.94,10.83,8.05,7.49,24.84,16.69,17.92,45.68,118.91,93.99,0.45,0.88,3.963095e+09,2.682301e+09,0.36,3.69,-9.68,2.30,-0.17,-0.62,-0.59,8.09,47.20,111.87,100.17,0.28,0.61,-1.268170e+08,3.725843e+09,74.60,20.05,42.48,8.60,7.44,37.52,25.24,29.18,32.45,208.15,134.93,0.29,0.78,7.710943e+09,3.268879e+09,0,0,0
1397,125028,046970,(주)우리로,42.14,8.83,92.45,4.43,-0.55,-2.82,-1.06,8.50,48.09,107.95,194.50,0.52,0.96,-3.391840e+08,2.333764e+09,-0.43,-19.09,217.55,0.32,-11.97,-67.22,-8.47,0.23,39.08,155.90,126.44,1.41,3.24,-8.603656e+09,1.597561e+09,-18.92,-30.55,-4.12,-0.39,-17.91,-85.22,-11.94,-0.26,33.47,198.77,51.52,1.50,4.10,-1.162556e+10,9.402770e+08,0,0,1
1398,383046,113810,디젠스(주),-12.94,6.17,-29.81,-3.98,-16.02,-87.40,-31.83,-7.91,38.61,158.99,63.05,0.50,1.44,-1.185408e+10,-3.781308e+09,2.65,9.68,6.59,3.77,3.12,13.36,5.49,6.63,41.25,142.41,78.82,0.57,1.42,2.180337e+09,2.726582e+09,-10.39,-27.27,24.96,-0.54,-11.28,-46.36,-15.25,-0.73,33.48,198.66,30.06,0.74,1.97,-7.563931e+09,1.054284e+09,0,0,0
1399,F11411,155650,와이엠씨(주),-22.34,13.57,59.47,10.09,7.46,196.38,6.33,8.55,50.08,99.67,800.95,1.18,2.87,1.464977e+10,-8.380533e+09,-27.33,-6.80,-29.90,-3.49,-1.69,-25.00,-1.54,-3.18,64.23,55.70,746.97,1.10,1.96,-2.496629e+09,-1.223279e+10,-13.46,5.52,-10.29,8.82,6.49,75.52,5.1

In [39]:
tmp_total.head(30)

,기업,연도,총자산증가율,자기자본증가율,매출액증가율,총자본영업이익율,총자본순이익율,매출액순이익율,매출액영업이익율,자기자본비율,부채비율,총자본회전율,자기자본회전율,당기순이익(손실)
0,006730,2018,14.780118,25.422116,87.868600,-0.000894,-0.601708,-9.599407,-0.014264,37.597608,165.974364,6.268175,16.671738,-1.037741e+10
1,006730,2019,5.018489,3.565213,18.345080,0.797204,0.002600,0.036810,11.286099,37.124510,169.363825,7.063591,19.026759,4.709363e+07
2,006730,2020,0.384633,-3.820188,-34.260713,-0.356007,-0.925143,-19.999794,-7.696186,35.621458,180.729669,4.625762,12.985887,-1.682075e+10
3,018700,2018,-4.879579,-5.590801,149.498529,-5.245811,-14.654924,-64.612484,-23.128394,93.148735,7.355189,22.681258,24.349507,-5.642155e+09
4,018700,2019,-3.492879,-5.101591,-59.848235,-8.936651,-19.045420,-201.826478,-94.702701,91.835017,8.890926,9.436532,10.275527,-7.076383e+09
5,018700,2020,37.768565,16.449343,602.879489,1.780440,-16.693827,-34.674703,3.698148,79.782622,25.340579,48.144109,60.344105,-8.545291e+09
6,019570,2018,NaN,NaN,NaN,-4.351808,-6.230429,NaN,NaN,85.395283,17.102487,NaN,NaN,-1.060834e+09
7,019570,2019,52.392984,27.867087,NaN,-12.870363,-12.568642,NaN,NaN,77.684690,28.725492,NaN,NaN,-3.261240e+09
8,019570,2020,125.392705,8.554741,NaN,-18.783385,-19.006111,NaN,NaN,37.690716,165.317327,NaN,NaN,-1.111546e+10
9,020400,2018,22.603525,16.574270,8.236393,2.240091,1.011523,1.064152,2.356640,38.904481,157.039799,95.054446,244.327758,8.605013e+08


In [41]:
df.to_csv('df_원본.csv', index=False)
tmp_total.to_csv('opendart.csv', index = False)

In [ ]:
tmp_total.describe()

,총자산증가율,자기자본증가율,매출액증가율,총자본영업이익율,총자본순이익율,매출액순이익율,매출액영업이익율,자기자본비율,부채비율,총자본회전율,자기자본회전율,당기순이익(손실)
count,88.000000,88.000000,68.000000,135.000000,135.000000,107.000000,107.000000,134.000000,134.000000,127.000000,126.000000,1.350000e+02
mean,42.507364,29.026544,2235.465013,-9.988864,-20.983598,-5667.677294,-4907.532830,58.127098,-167.832422,32.639211,-244.681986,-8.173414e+09
std,112.720782,177.253779,18163.158579,22.763095,59.103207,35439.949420,34309.035488,88.286598,1626.502299,55.956746,2233.421860,1.770613e+10
min,-82.551375,-126.747407,-100.000000,-118.470090,-622.358107,-353205.536152,-350051.850380,-690.981106,-16075.124467,0.000000,-21213.969911,-1.172341e+11
25%,-4.240969,-5.223894,-33.161686,-16.802564,-22.953491,-363.085580,-261.718893,46.867990,6.583570,0.442905,0.236047,-1.120089e+10
50%,3.784697,3.963705,1.416480,-4.462285,-9.525771,-34.674703,-11.225950,76.605119,20.027772,13.113658,13.315242,-4.045846e+09
75%,38.066292,21.201596,35.998177,1.479374,0.309771,-0.045691,6.303471,92.566304,81.943026,41.693085,49.508001,4.302014e+07
max,714.365493,1581.318537,149805.530667,58.943951,44.786053,309.991346,80.554352,99.635646,607.021922,477.772352,860.731007,2.663508e+10


In [ ]:
df.describe()

,총자산증가율 (2018),자기자본증가율 (2018),매출액증가율 (2018),총자본영업이익율 (2018),총자본순이익율 (2018),자본금순이익율 (2018),매출액순이익율 (2018),매출액영업이익율 (2018),자기자본비율 (2018),부채비율 (2018),유보액/납입자본비율 (2018),총자본회전율 (2018),자기자본회전율 (2018),당기순이익(손실) (2018),영업활동으로인한현금흐름 (2018),총자산증가율 (2019),자기자본증가율 (2019),매출액증가율 (2019),총자본영업이익율 (2019),총자본순이익율 (2019),자본금순이익율 (2019),매출액순이익율 (2019),매출액영업이익율 (2019),자기자본비율 (2019),부채비율 (2019),유보액/납입자본비율 (2019),총자본회전율 (2019),자기자본회전율 (2019),당기순이익(손실) (2019),영업활동으로인한현금흐름 (2019),총자산증가율 (2020),자기자본증가율 (2020),매출액증가율 (2020),총자본영업이익율 (2020),총자본순이익율 (2020),자본금순이익율 (2020),매출액순이익율 (2020),매출액영업이익율 (2020),자기자본비율 (2020),부채비율 (2020),유보액/납입자본비율 (2020),총자본회전율 (2020),자기자본회전율 (2020),당기순이익(손실) (2020),영업활동으로인한현금흐름 (2020),2019_관리종목여부,2020_관리종목여부,2021_관리종목여부
count,1.385000e+03,1358.000000,1367.000000,1400.000000,1401.000000,1401.000000,1384.000000,1384.000000,1382.000000,1382.000000,1401.000000,1375.000000,1359.000000,1.378000e+03,1.368000e+03,1399.000000,1364.000000,1381.000000,1400.000000,1399.000000,1401.000000,1388.000000,1389.000000,1380.000000,1380.000000,1401.000000,1382.000000,1354.000000,1.378000e+03,1.377000e+03,1398.000000,1369.000000,1387.000000,1401.000000,1401.000000,1401.000000,1393.000000,1393.000000,1384.000000,1384.000000,1401.000000,1384.000000,1361.000000,1.378000e+03,1.378000e+03,1401.000000,1401.000000,1401.000000
mean,8.307525e+02,685.906127,21.573921,2.502879,-1.536881,70.205332,-616.313439,-471.265383,64.496852,97.685753,1697.744404,0.802269,1.610412,2.820362e+09,5.502024e+09,21.805661,32.059802,116.543526,1.947186,-2.671001,79.691863,-372.374265,-111.402441,64.173964,98.897138,1533.109436,0.749609,1.467866,2.259509e+09,7.875689e+09,18.942747,56.604580,127.856676,2.581556,-1.579579,24.086110,-172.355441,-123.380158,65.543338,79.263967,1267.474233,0.694299,1.296084,3.913999e+09,1.034705e+10,0.029979,0.027123,0.020700
std,2.926893e+04,23315.797840,153.383154,15.853033,26.836112,2323.153378,11080.499725,10273.463568,20.955324,299.279136,9641.966500,0.633225,2.491764,1.698619e+10,1.958206e+10,77.475601,173.069248,3535.374771,15.856586,25.873139,2077.977273,6496.959176,1348.932879,20.868991,353.541759,8576.863583,0.573866,1.941010,2.055404e+10,2.332077e+10,82.939257,938.780965,4023.659353,14.783134,21.454284,1207.420503,2855.134204,1830.124324,19.833842,136.178217,2591.628335,0.522568,1.436657,4.674366e+10,3.025509e+10,0.170589,0.162501,0.142427
min,-6.804000e+01,-90.030000,-100.000000,-129.530000,-532.050000,-64738.350000,-353205.520000,-350051.850000,1.120000,0.910000,-8327.170000,0.010000,0.010000,-1.234088e+11,-1.672801e+11,-82.480000,-98.300000,-100.000000,-191.540000,-353.270000,-25660.800000,-184682.910000,-32961.540000,1.080000,0.430000,-17899.570000,0.010000,0.010000,-2.129300e+11,-1.452495e+11,-68.780000,-84.860000,-100.000000,-114.110000,-263.500000,-30718.010000,-90644.630000,-50203.430000,3.050000,0.370000,-51218.030000,0.010000,0.010000,-9.611484e+10,-6.483745e+10,0.000000,0.000000,0.000000
25%,-1.530000e+00,-1.100000,-8.145000,-0.775000,-3.400000,-31.970000,-4.920000,-0.890000,49.792500,21.345000,334.930000,0.400000,0.580000,-2.380625e+09,-1.445045e+09,-1.220000,-1.172500,-8.560000,-0.417500,-3.975000,-35.650000,-7.087500,-0.550000,49.500000,22.012500,336.420000,0.390000,0.550000,-2.758331e+09,-7.153330e+08,-2.560000,-1.500000,-13.415000,-1.410000,-5.300000,-48.280000,-10.630000,-2.890000,51.880000,22.022500,353.050000,0.340000,0.490000,-4.360709e+09,-6.530748e+08,0.000000,0.000000,0.000000
50%,7.340000e+00,6.950000,4.380000,2.905000,2.620000,33.020000,3.670000,4.455000,65.125000,53.550000,783.650000,0.660000,1.050000,1.959088e+09,2.313819e+09,7.260000,5.500000,5.300000,2.810000,2.210000,28.120000,3.070000,4.470000,65.405000,52.885000,806.530000,0.620000,1.000000,1.810330e+09,3.279718e+09,5.925000,5.750000,1.930000,2.480000,1.900000,25.840000,2.960000,4.320000,67.745000,47.610000,839.530000,0.590000,0.920000,1.633597e+09,4.268566e+09,0.000000,0.000000,0.000000
75%,2.496000e+01,23.480000,20.830000,8.742500,7.760000,143.810000,10.50000

In [ ]:
# 채우기 이전에 df의 널값 총 개수
print(df.isna().sum().sum())

734


In [ ]:
# # 널값인 데이터만 보기
# df2 = df.copy()
# df2['NaN_cnt'] = df2.isnull().sum(1)

# # 재무제표 crawling이 필요한 코드
# stock_code = list(df2[df2['NaN_cnt']>0]['Stock'].values)

# 연도별로 나누기
df_base = df[['KIS', 'Stock', 'Name']]
df_2018 = pd.concat([df_base,df[[col for col in df.columns if '2018' in col]]], axis=1)
df_2019 = pd.concat([df_base,df[[col for col in df.columns if '2019' in col]]], axis=1)
df_2020 = pd.concat([df_base,df[[col for col in df.columns if '2020' in col]]], axis=1)
  
# 채우기 위해 크롤링한 open Dart df
temp_df_2018 = tmp_total[tmp_total['연도'] == 2018].reset_index(drop=True)
temp_df_2019 = tmp_total[tmp_total['연도'] == 2019].reset_index(drop=True)
temp_df_2020 = tmp_total[tmp_total['연도'] == 2020].reset_index(drop=True)


# 채우기 이전에 null값 개수
print(f'2018년도 채우기 이전 널값 개수: {df_2018.isna().sum().sum()}')
print(f'2019년도 채우기 이전 널값 개수: {df_2019.isna().sum().sum()}')
print(f'2020년도 채우기 이전 널값 개수: {df_2020.isna().sum().sum()}')

# df_2018.replace({np.NAN: 'none'}, inplace=True)

# # 채우기
# for i in range(len(df_2018)):
#   for j in df_2018.columns[3:]:
#     if df_2018.loc[i, j]== 'none':
#       stock_ccode = df_2018['Stock'][i]
#       try:
#         df_2018.loc[i,j]=
#       except:
#         pass

# df_2018.replace({'none', np.NAN}, inplace=True)

2018년도 채우기 이전 널값 개수: 290
2019년도 채우기 이전 널값 개수: 242
2020년도 채우기 이전 널값 개수: 202


In [ ]:
df_2018.head(30)

,KIS,Stock,Name,총자산증가율 (2018),자기자본증가율 (2018),매출액증가율 (2018),총자본영업이익율 (2018),총자본순이익율 (2018),자본금순이익율 (2018),매출액순이익율 (2018),매출액영업이익율 (2018),자기자본비율 (2018),부채비율 (2018),유보액/납입자본비율 (2018),총자본회전율 (2018),자기자본회전율 (2018),당기순이익(손실) (2018),영업활동으로인한현금흐름 (2018)
0,036644,389500,(주)에스비비테크,46.03,-39.46,2.26,-5.23,-9.21,-168.76,-20.80,-11.80,20.22,394.44,340.07,0.44,1.39,-1.917031e+09,6.075800e+07
1,497730,296640,(주)이노룰스,81.84,215.02,-6.23,3.88,1.43,68.58,1.90,5.16,40.95,144.17,2371.70,0.75,2.16,1.770280e+08,1.148444e+09
2,499459,417970,모델솔루션(주),23.53,44.46,9.65,25.30,19.34,768.48,12.97,16.97,47.94,108.60,2005.45,1.49,3.33,7.210181e+09,5.469434e+09
3,HV2035,393890,더블유씨피(주),69.90,68.28,NaN,-8.72,-7.59,-559.62,NaN,NaN,93.30,7.18,8560.18,NaN,NaN,-9.513513e+09,-8.218473e+09
4,HS8995,314140,(주)알피바이오,38.75,31.69,-5.02,8.28,5.66,146.00,6.74,9.85,58.82,70.00,1558.22,0.84,1.40,4.363446e+09,6.995499e+09
5,389238,368970,(주)오에스피,NaN,NaN,NaN,21.66,15.28,284.44,9.37,13.29,54.28,84.25,910.65,1.63,3.00,1.137761e+09,NaN
6,IW9267,394280,오픈엣지테크놀로지(주),24565.93,24767.99,NaN,26.41,25.40,2683.41,36.49,37.94,93.82,6.59,17064.75,0.70,0.74,4.735680e+08,NaN
7,484917,148930,(주)에이치와이티씨,6.46,NaN,48.83,21.61,14.00,74.47,10.01,15.46,NaN,NaN,-190.55,1.40,NaN,1.940563e+09,4.664951e+09
8,894281,107600,(주)새빗켐,24.53,15.87,5.81,14.03,10.14,51.67,7.86,10.88,54.11,84.81,205.99,1.29,2.31,1.058424e+09,NaN
9,867055,129920,(주)대성하이텍,8.36,26.30,7.67,3.96,1.57,162.88,2.10,5.32,38.03,162.97,4012.35,0.74,2.10,1.865670e+09,8.755060e+08


In [ ]:
# 널값인 데이터만 보기
df22 = df_2018.copy()
df22['NaN_cnt'] = df22.isnull().sum(1)

# 재무제표 crawling이 필요한 코드
stock_code_2018 = list(df22[df22['NaN_cnt']>0]['Stock'].values)
stock_code_2018


tmp_2018 = []
for i in stock_code_2018:
  if i in list(temp_df_2018['기업'].unique()):
    tmp_2018.append(i)
print(tmp_2018)

['299660', '307180', '309930', '298380', '290660', '208340', '115180', '042420', '042370', '006730', '023460', '021080', '019570', '270520', '277070', '291230', '284620']


In [44]:
df22 = df.copy()
df22['NaN_cnt'] = df22.isnull().sum(1)

# 재무제표 crawling이 필요한 코드
stock_code_null = list(df22[df22['NaN_cnt']>0]['Stock'].values)
stock_code_null

['393890',
 '368970',
 '394280',
 '148930',
 '107600',
 '397030',
 '328130',
 '310210',
 '362320',
 '389140',
 '388050',
 '376930',
 '330730',
 '276040',
 '298870',
 '377480',
 '357880',
 '376300',
 '389030',
 '261780',
 '308080',
 '315640',
 '376980',
 '195940',
 '377030',
 '365270',
 '352910',
 '187660',
 '363250',
 '347700',
 '338220',
 '334970',
 '340930',
 '321820',
 '236810',
 '335810',
 '354200',
 '317690',
 '348150',
 '304840',
 '331920',
 '950200',
 '330350',
 '339950',
 '288330',
 '235980',
 '335890',
 '190650',
 '216080',
 '317240',
 '100790',
 '299660',
 '314130',
 '307180',
 '310870',
 '309930',
 '298380',
 '307750',
 '307870',
 '291650',
 '294090',
 '323990',
 '298060',
 '290660',
 '027360',
 '208340',
 '306620',
 '244460',
 '300080',
 '293780',
 '322780',
 '307930',
 '319400',
 '227950',
 '217730',
 '115180',
 '228760',
 '239890',
 '241820',
 '241520',
 '246690',
 '267320',
 '027830',
 '054220',
 '058450',
 '007330',
 '016600',
 '036170',
 '037460',
 '042420',
 '042370',

In [46]:
print(len(stock_code_null))

117


In [47]:
len(df)

1401

In [ ]:
# 기업 연도 한개씩으로 내가 새로 만들어보기
# 정민이한테 물어보기
stock_code = '284620'
year = 2018
indicators = ['자산총계', '부채총계', '자본총계', '매출액', '영업이익', '당기순이익', '법인세차감전 순이익']


result = dart.finstate(stock_code, year,'11011') # 사업보고서 데이터 가져오기
# if report is None: # 리포트가 없다면 (참고 : 리포트가 없으면 None을 반환함)
result = result.loc[result['fs_nm'] =='재무제표'] # 연결재무제표 아닌 재무제표 필터링
result = result.loc[result['account_nm'].isin(indicators)]
data = [] 
for y, c in zip([year], ['thstrm_amount']):
  record = [stock_code, y]
  for indic in indicators:
    # account_nm이 indic인 행의 c 컬럼 값을 가져옴
    if sum(result['account_nm']==indic)>0: # 해당 지표가 있다면 추가(지표가 없는 경우가 있음)
      value = result.loc[result['account_nm']==indic, c].iloc[0]
    else:
      value = np.nan

    record.append(value)

  data.append(record)
tt= pd.DataFrame(data, columns = ['기업', '연도'] + indicators)  
tt
result

,rcept_no,reprt_code,bsns_year,corp_code,stock_code,fs_div,fs_nm,sj_div,sj_nm,account_nm,thstrm_nm,thstrm_dt,thstrm_amount,frmtrm_nm,frmtrm_dt,frmtrm_amount,bfefrmtrm_amount,ord,currency
2,20190401004592,11011,2018,01263527,284620,OFS,재무제표,BS,재무상태표,자산총계,제 2 기,2018.12.31 현재,"10,599,449,914",제 1 기,2017.12.31 현재,"1,690,775,632",-,6,KRW
5,20190401004592,11011,2018,01263527,284620,OFS,재무제표,BS,재무상태표,부채총계,제 2 기,2018.12.31 현재,"772,060,139",제 1 기,2017.12.31 현재,"740,190,004",-,12,KRW
8,20190401004592,11011,2018,01263527,284620,OFS,재무제표,BS,재무상태표,자본총계,제 2 기,2018.12.31 현재,"9,827,389,775",제 1 기,2017.12.31 현재,"950,585,628",-,22,KRW
9,20190401004592,11011,2018,01263527,284620,OFS,재무제표,IS,손익계산서,매출액,제 2 기,2018.01.01 ~ 2018.12.31,0,제 1 기,2017.11.03 ~ 2017.12.31,0,-,24,KRW
10,20190401004592,11011,2018,01263527,284620,OFS,재무제표,IS,손익계산서,영업이익,제 2 기,2018.01.01 ~ 2018.12.31,"-25,917,490",제 1 기,2017.11.03 ~ 2017.12.31,"-7,044,640",-,26,KRW
11,20190401004592,11011,2018,01263527,284620,OFS,재무제표,IS,손익계산서,법인세차감전 순이익,제 2 기,2018.01.01 ~ 2018.12.31,"49,931,598",제 1 기,2017.11.03 ~ 2017.12.31,"-9,495,262",-,28,KRW
12,20190401004592,11011,2018,01263527,284620,OFS,재무제표,IS,손익계산서,당기순이익,제 2 기,2018.01.01 ~ 2018.12.31,"38,946,647",제 1 기,2017.11.03 ~ 2017.12.31,"-7,406,305",-,30,KRW


In [ ]:
temp_df_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   기업         31 non-null     object 
 1   연도         31 non-null     object 
 2   총자산증가율     20 non-null     float64
 3   자기자본증가율    20 non-null     float64
 4   매출액증가율     18 non-null     float64
 5   총자본영업이익율   31 non-null     float64
 6   총자본순이익율    31 non-null     float64
 7   매출액순이익율    23 non-null     float64
 8   매출액영업이익율   23 non-null     float64
 9   자기자본비율     31 non-null     float64
 10  부채비율       31 non-null     float64
 11  총자본회전율     29 non-null     float64
 12  자기자본회전율    29 non-null     float64
 13  당기순이익(손실)  31 non-null     float64
dtypes: float64(12), object(2)
memory usage: 3.5+ KB


In [ ]:
df_2018.loc[0,'총자산증가율 (2018)']

46.03

In [ ]:
temp_df_2018.columns

Index(['기업', '연도', '총자산증가율', '자기자본증가율', '매출액증가율', '총자본영업이익율', '총자본순이익율',
       '매출액순이익율', '매출액영업이익율', '자기자본비율', '부채비율', '총자본회전율', '자기자본회전율',
       '당기순이익(손실)'],
      dtype='object')

In [ ]:
df_2018.columns

Index(['KIS', 'Stock', 'Name', '총자산증가율 (2018)', '자기자본증가율 (2018)',
       '매출액증가율 (2018)', '총자본영업이익율 (2018)', '총자본순이익율 (2018)', '자본금순이익율 (2018)',
       '매출액순이익율 (2018)', '매출액영업이익율 (2018)', '자기자본비율 (2018)', '부채비율 (2018)',
       '유보액/납입자본비율 (2018)', '총자본회전율 (2018)', '자기자본회전율 (2018)',
       '당기순이익(손실) (2018)', '영업활동으로인한현금흐름 (2018)'],
      dtype='object')

In [ ]:
print(tmp_total.shape)
tmp_total.head(30)


(218, 13)


,기업,연도,총자산증가율,자기자본증가율,매출액증가율,총자본영업이익율,총자본순이익율,매출액순이익율,매출액영업이익율,자기자본비율,부채비율,총자본회전율,자기자본회전율
0,006730,2018,-0.668837,-2.689208,87.868600,-0.000894,-0.601708,-9.599407,-0.014264,37.597608,165.974364,6.268175,16.671738
1,006730,2019,5.018489,3.565213,18.345080,0.797204,0.002600,0.036810,11.286099,37.124510,169.363825,7.063591,19.026759
2,006730,2020,9.260552,-6.685650,-33.750694,-1.145787,-1.779551,-41.549607,-26.752264,31.848672,213.984832,4.282955,13.447828
3,018700,2018,-3.934614,-34.399102,42.034220,-9.445938,-27.292588,-163.494957,-56.585444,68.824122,45.297895,16.693229,24.254911
4,018700,2018,6.588345,12.502938,47.506839,-6.846527,-13.681486,-59.223010,-29.636542,73.796793,35.507244,23.101639,31.304395
5,018700,2019,31.469350,35.435777,-40.431197,-4.871750,-24.928192,-238.151662,-46.542302,86.940282,15.021481,10.467360,12.039712
6,018700,2019,-28.638205,-77.016848,13.806998,-18.286298,-27.292588,-163.494957,-109.543203,68.824122,45.297895,16.693229,24.254911
7,018700,2020,31.864254,33.953258,299.544877,0.101342,-17.343509,-34.289260,0.200359,79.024580,26.542906,50.580002,64.005405
8,019570,2018,2.916121,-7.778908,NaN,-4.351808,-6.230429,NaN,NaN,85.395283,17.102487,NaN,NaN
9,019570,2019,52.392984,27.867087,NaN,-12.870363,-12.568642,NaN,NaN,77.684690,28.725492,NaN,NaN


# 최종 파일 만들기
- nan이 있는 종목 drop하기
- nan이 있는 데이터 117개 drop
- 1284개 기업 데이터로 확정

In [51]:
df22 = df.copy()
print(df22.isna().sum().sum())
del_list = list(df2[df2['NaN_cnt']>0].index)

734


In [52]:
df22.drop(del_list, axis=0, inplace=True)
df22.isna().sum()

KIS                    0
Stock                  0
Name                   0
총자산증가율 (2018)          0
자기자본증가율 (2018)         0
매출액증가율 (2018)          0
총자본영업이익율 (2018)        0
총자본순이익율 (2018)         0
자본금순이익율 (2018)         0
매출액순이익율 (2018)         0
매출액영업이익율 (2018)        0
자기자본비율 (2018)          0
부채비율 (2018)            0
유보액/납입자본비율 (2018)      0
총자본회전율 (2018)          0
자기자본회전율 (2018)         0
당기순이익(손실) (2018)       0
영업활동으로인한현금흐름 (2018)    0
총자산증가율 (2019)          0
자기자본증가율 (2019)         0
매출액증가율 (2019)          0
총자본영업이익율 (2019)        0
총자본순이익율 (2019)         0
자본금순이익율 (2019)         0
매출액순이익율 (2019)         0
매출액영업이익율 (2019)        0
자기자본비율 (2019)          0
부채비율 (2019)            0
유보액/납입자본비율 (2019)      0
총자본회전율 (2019)          0
자기자본회전율 (2019)         0
당기순이익(손실) (2019)       0
영업활동으로인한현금흐름 (2019)    0
총자산증가율 (2020)          0
자기자본증가율 (2020)         0
매출액증가율 (2020)          0
총자본영업이익율 (2020)        0
총자본순이익율 (2020)         0
자본금순이익율 (2020)         0
매출액순이익율 (2020)         0


In [53]:
len(df22)

1284

In [54]:
print(df22['2019_관리종목여부'].value_counts())
print(df22['2020_관리종목여부'].value_counts())
print(df22['2021_관리종목여부'].value_counts())

0    1245
1      39
Name: 2019_관리종목여부, dtype: int64
0    1252
1      32
Name: 2020_관리종목여부, dtype: int64
0    1259
1      25
Name: 2021_관리종목여부, dtype: int64


In [56]:
# 최종파일 내보내기
df22.reset_index(drop=True, inplace= True)
df22.to_csv('real_df.csv', index=False)

In [ ]:
# https://gils-lab.tistory.com/38?category=504501
